In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models
import pandas as pd
import numpy as np
from PIL import Image
import os
from sklearn.metrics import precision_recall_fscore_support
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import wandb
from tqdm import tqdm
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
print('imported')

imported


In [2]:
import wandb
import os
wandb.login(key='02f62ea18d807de380f948a102283d73ce32d0ef')

os.environ["WANDB_MODE"] = "online"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mohiuddinprantiq (mohiuddinprantiq-chittagong-university-of-engineering-an). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
class MemeImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.data.iloc[idx]['image_id']}.jpg")
        image = Image.open(img_name).convert('RGB')
        label = self.data.iloc[idx]['labels']
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.long)


In [4]:
def get_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])


In [5]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5, model_name=""):
    wandb.init(project="misogyny-detection", name=model_name, config={"settings": {"console": "off"}})
    best_val_f1 = 0.0
    best_model_weights = None
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        train_preds, train_labels = [], []
        
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            if isinstance(outputs, torch.Tensor):
                logits = outputs
            else:  # For transformers models
                logits = outputs.logits
                
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(logits, 1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        # Calculate training metrics
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(
            train_labels, train_preds, average='binary'
        )
        
        # Validation phase
        model.eval()
        val_preds, val_labels = [], []
        val_loss = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                if isinstance(outputs, torch.Tensor):
                    logits = outputs
                else:
                    logits = outputs.logits
                
                loss = criterion(logits, labels)
                val_loss += loss.item()
                
                _, preds = torch.max(logits, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        # Calculate validation metrics
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(
            val_labels, val_preds, average='binary'
        )
        
        # Log metrics
        wandb.log({
            'epoch': epoch+1,
            'train_loss': running_loss/len(train_loader),
            'val_loss': val_loss/len(val_loader),
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall
        })
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {running_loss/len(train_loader):.4f}')
        print(f'Train F1: {train_f1:.4f}')
        print(f'val Loss: {val_loss/len(val_loader):.4f}')
        print(f'Val F1: {val_f1:.4f}')
        
        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_weights = model.state_dict().copy()
    
    # Load best model weights
    model.load_state_dict(best_model_weights)
    wandb.finish()
    return model


In [6]:
def evaluate_model(model, test_loader, device):
    """Evaluate the model"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            if isinstance(outputs, torch.Tensor):
                logits = outputs
            else:
                logits = outputs.logits
            
            _, preds = torch.max(logits, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary'
    )
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [7]:
def get_model(model_name):
    """Initialize model based on name"""
    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        model.classifier[-1] = nn.Linear(4096, 2)
    elif model_name == 'vgg19':
        model = models.vgg19(pretrained=True)
        model.classifier[-1] = nn.Linear(4096, 2)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(2048, 2)
    elif model_name == 'vit':
        model = ViTForImageClassification.from_pretrained(
            'google/vit-base-patch16-224',
            num_labels=2,
            ignore_mismatched_sizes=True
        )
    elif model_name == 'swin':
        model = AutoModelForImageClassification.from_pretrained(
            'microsoft/swin-tiny-patch4-window7-224',
            num_labels=2,
            ignore_mismatched_sizes=True
        )
    return model


In [8]:
def main(train_csv, val_csv, train_img_dir, val_img_dir, test_csv, test_img_dir, lan):
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create datasets
    transform = get_transforms()
    train_dataset = MemeImageDataset(train_csv, train_img_dir, transform=transform)
    val_dataset = MemeImageDataset(val_csv, val_img_dir, transform=transform)
    test_dataset = MemeImageDataset(test_csv, test_img_dir, transform=transform)
    
    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Models to evaluate
    models_to_train = ['vgg16', 'vgg19', 'resnet50', 'vit', 'swin']
    results = {}
    
    for model_name in models_to_train:
        print(f"\nTraining {model_name}...")
        model = get_model(model_name)
        model = model.to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        
        # Train
        model = train_model(
            model, train_loader, val_loader, criterion, optimizer, device,
            num_epochs=5, model_name=model_name
        )
        
        # Evaluate
        metrics = evaluate_model(model, test_loader, device)
        results[model_name] = metrics
        
        # Save model
        torch.save(model.state_dict(), f'{model_name}_best.pth')
    
    # Print final comparison
    rows = []
    for model_name, metrics in results.items():
        rows.append([
            model_name,
            f"{metrics['precision']:.4f}",
            f"{metrics['recall']:.4f}",
            f"{metrics['f1']:.4f}"
        ])
    headers = ['Model', 'Precision', 'Recall', 'F1-Score']
    print(f"\nModel Comparison Results for {lan}:")
    print(tabulate(rows, headers=headers, tablefmt='grid'))

    # Save results to CSV
    results_df = pd.DataFrame(rows, columns=headers)
    results_df.to_csv(f'model_comparison_results_{lan}.csv', index=False)
    print(f"\nResults saved to 'model_comparison_results_for_{lan}.csv'")

In [9]:
# Malayalam
train_csv = '/kaggle/input/misogyny/misogyny/misogyny/malayalam/train/train.csv'
val_csv = '/kaggle/input/misogyny/misogyny/misogyny/malayalam/dev/dev.csv'
    
train_img_dir = '/kaggle/input/misogyny/misogyny/misogyny/malayalam/train/memes'
val_img_dir = '/kaggle/input/misogyny/misogyny/misogyny/malayalam/dev/memes'

test_csv = '/kaggle/input/misogyny/test_with_labels_malayalam/test_with_labels.csv'
test_img_dir = '/kaggle/input/misogyny/test_with_labels_malayalam/memes'
main(train_csv, val_csv, train_img_dir, val_img_dir, test_csv, test_img_dir, 'malayalam')

Using device: cuda

Training vgg16...


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 224MB/s]  


Epoch 1/5: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it]


Epoch 1/5:
Train Loss: 0.5534
Train F1: 0.6653
val Loss: 0.5280
Val F1: 0.6800


Epoch 2/5: 100%|██████████| 20/20 [00:18<00:00,  1.07it/s]


...

Epoch 4/5:
Train Loss: 0.0416
Train F1: 0.9864
val Loss: 0.5109
Val F1: 0.8226


Epoch 5/5: 100%|██████████| 20/20 [00:17<00:00,  1.11it/s]


Epoch 5/5:
Train Loss: 0.0118
Train F1: 0.9981
val Loss: 0.6488
Val F1: 0.8099


epoch,▁▃▅▆█
train_f1,▁▇▇██
train_loss,█▃▃▁▁
train_precision,▁▇▇██
train_recall,▁▆▇██
val_f1,▃▇▁█▇
val_loss,▁▂▃▄█
val_precision,▃█▁▇▇
val_recall,▄▁█▄▂
epoch,5
train_f1,0.99807



Model Comparison Results for malayalam:
+----------+-------------+----------+------------+
| Model    |   Precision |   Recall |   F1-Score |
+==========+=============+==========+============+
| vgg16    |      0.7765 |   0.8462 |     0.8098 |
+----------+-------------+----------+------------+
| vgg19    |      0.9138 |   0.6795 |     0.7794 |
+----------+-------------+----------+------------+
| resnet50 |      0.75   |   0.7692 |     0.7595 |
+----------+-------------+----------+------------+
| vit      |      0.9057 |   0.6154 |     0.7328 |
+----------+-------------+----------+------------+
| swin     |      0.8676 |   0.7564 |     0.8082 |
+----------+-------------+----------+------------+

Results saved to 'model_comparison_results_for_malayalam.csv'


In [10]:
# Tamil
train_csv = '/kaggle/input/misogyny/misogyny/misogyny/tamil/train/train.csv'
val_csv = '/kaggle/input/misogyny/misogyny/misogyny/tamil/dev/dev.csv'
    
train_img_dir = '/kaggle/input/misogyny/misogyny/misogyny/tamil/train/memes'
val_img_dir = '/kaggle/input/misogyny/misogyny/misogyny/tamil/dev/memes'

test_csv = '/kaggle/input/misogyny/test_with_labels_tamil/test_with_labels.csv'
test_img_dir = '/kaggle/input/misogyny/test_with_labels_tamil/memes'
main(train_csv, val_csv, train_img_dir, val_img_dir, test_csv, test_img_dir, 'tamil')

Using device: cuda

Training vgg16...


Epoch 1/5: 100%|██████████| 36/36 [00:29<00:00,  1.22it/s]


Epoch 1/5:
Train Loss: 0.4962
Train F1: 0.2972
val Loss: 0.4836
Val F1: 0.5103


Epoch 2/5: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s]


Epoch 2/5:
Train Loss: 0.3107
Train F1: 0.6693
val Loss: 0.4558
Val F1: 0.5294


...

Epoch 4/5:
Train Loss: 0.0522
Train F1: 0.9700
val Loss: 0.9360
Val F1: 0.4660


Epoch 5/5: 100%|██████████| 36/36 [00:23<00:00,  1.56it/s]


Epoch 5/5:
Train Loss: 0.0563
Train F1: 0.9700
val Loss: 0.7039
Val F1: 0.5760


epoch,▁▃▅▆█
train_f1,▁▅▇██
train_loss,█▅▃▁▁
train_precision,▁▄▇██
train_recall,▁▅▇██
val_f1,▁██▄█
val_loss,▁▁▃█▅
val_precision,▁▆▅█▂
val_recall,▁▇█▃█
epoch,5
train_f1,0.97002



Model Comparison Results for tamil:
+----------+-------------+----------+------------+
| Model    |   Precision |   Recall |   F1-Score |
+==========+=============+==========+============+
| vgg16    |      0.7656 |   0.5506 |     0.6405 |
+----------+-------------+----------+------------+
| vgg19    |      0.8462 |   0.1236 |     0.2157 |
+----------+-------------+----------+------------+
| resnet50 |      0.7568 |   0.6292 |     0.6871 |
+----------+-------------+----------+------------+
| vit      |      0.7647 |   0.5843 |     0.6624 |
+----------+-------------+----------+------------+
| swin     |      0.7368 |   0.6292 |     0.6788 |
+----------+-------------+----------+------------+

Results saved to 'model_comparison_results_for_tamil.csv'
